In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from sklearn.preprocessing import LabelEncoder # 라벨 인코더
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings('ignore')

In [4]:
from bayes_opt import BayesianOptimization

In [5]:
final = pd.read_csv('/Users/sungahwang/Desktop/BC_data/final.csv')
final.head()

,user_id,sum_applied,application_id,gender,insert_time,yearly_income,income_type,employment_type,houseown_type,desired_amount,...,is_applied,weekday,loanapply_insert_hour,bank_apply_rate,product_apply_rate,cluster2,action_cnt,use_day_cnt,first_event,timeout_sum(s)
0,545882.0,8.0,576643,1.0,2022-05-09 10:54:51,72000000.0,직장가입자(4대보험O),정규직,전월세,20000000.0,...,0.0,0,10,0.042501,0.050284,0,41.913806,5.765978,CompleteIDCertification,67.997282
1,545882.0,8.0,576643,1.0,2022-05-09 10:54:51,72000000.0,직장가입자(4대보험O),정규직,전월세,20000000.0,...,0.0,0,10,0.017018,0.016149,0,41.913806,5.765978,CompleteIDCertification,67.997282
2,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.024636,0.028622,0,41.913806,5.765978,CompleteIDCertification,67.997282
3,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.021634,0.035367,0,41.913806,5.765978,CompleteIDCertification,67.997282
4,558819.0,5.0,2136706,1.0,2022-05-09 10:41:05,39000000.0,직장가입자(4대보험O),정규직,전월세,80000000.0,...,0.0,0,10,0.027749,0.035265,4,41.913806,5.765978,CompleteIDCertification,67.997282


In [6]:
final = final.drop(['user_id', 'application_id', 'gender', 'insert_time', 'yearly_income', 'income_type', 'employment_type',
                 'houseown_type', 'desired_amount', 'purpose', 'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
                 'existing_loan_cnt', 'existing_loan_amt','reage','career','loanapply_insert_time'], axis = 1)

In [7]:
final.isnull().sum()

sum_applied                    0
credit_grade                   0
spec_clust                     0
bank_id                        0
product_id                     0
loan_limit                     0
loan_rate                      0
is_applied               3255482
weekday                        0
loanapply_insert_hour          0
bank_apply_rate                0
product_apply_rate             0
cluster2                       0
action_cnt                     0
use_day_cnt                    0
first_event                    0
timeout_sum(s)                 0
dtype: int64

In [8]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13141813 entries, 0 to 13141812
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   sum_applied            float64
 1   credit_grade           float64
 2   spec_clust             float64
 3   bank_id                int64  
 4   product_id             int64  
 5   loan_limit             float64
 6   loan_rate              float64
 7   is_applied             float64
 8   weekday                int64  
 9   loanapply_insert_hour  int64  
 10  bank_apply_rate        float64
 11  product_apply_rate     float64
 12  cluster2               int64  
 13  action_cnt             float64
 14  use_day_cnt            float64
 15  first_event            object 
 16  timeout_sum(s)         float64
dtypes: float64(11), int64(5), object(1)
memory usage: 1.7+ GB


In [9]:
final['spec_clust'] = final['spec_clust'].astype('str')
final['weekday'] = final['weekday'].astype('str')
final['cluster2'] = final['cluster2'].astype('str')

In [10]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13141813 entries, 0 to 13141812
Data columns (total 17 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   sum_applied            float64
 1   credit_grade           float64
 2   spec_clust             object 
 3   bank_id                int64  
 4   product_id             int64  
 5   loan_limit             float64
 6   loan_rate              float64
 7   is_applied             float64
 8   weekday                object 
 9   loanapply_insert_hour  int64  
 10  bank_apply_rate        float64
 11  product_apply_rate     float64
 12  cluster2               object 
 13  action_cnt             float64
 14  use_day_cnt            float64
 15  first_event            object 
 16  timeout_sum(s)         float64
dtypes: float64(10), int64(3), object(4)
memory usage: 1.7+ GB


In [11]:
final = pd.get_dummies(final)
final

,sum_applied,credit_grade,bank_id,product_id,loan_limit,loan_rate,is_applied,loanapply_insert_hour,bank_apply_rate,product_apply_rate,...,cluster2_1,cluster2_2,cluster2_3,cluster2_4,cluster2_5,first_event_CompleteIDCertification,first_event_EndLoanApply,first_event_GetCreditInfo,first_event_StartLoanApply,first_event_others
0,8.0,7.0,54,235,11000000.0,16.5,0.0,10,0.042501,0.050284,...,0,0,0,0,0,1,0,0,0,0
1,8.0,7.0,11,118,3000000.0,20.0,0.0,10,0.017018,0.016149,...,0,0,0,0,0,1,0,0,0,0
2,5.0,5.0,42,216,10000000.0,13.5,0.0,10,0.024636,0.028622,...,0,0,0,0,0,1,0,0,0,0
3,5.0,5.0,25,169,22000000.0,15.9,0.0,10,0.021634,0.035367,...,0,0,0,0,0,1,0,0,0,0
4,5.0,5.0,22,221,10000000.0,18.4,0.0,10,0.027749,0.035265,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13141808,4.0,3.0,60,183,60000000.0,5.2,1.0,13,0.112861,0.120308,...,0,0,0,0,0,1,0,0,0,0
13141809,14.0,6.0,2,7,30000000.0,11.8,1.0,14,0.098055,0.093935,...,0,0,0,0,0,1,0,0,0,0
13141810,6.0,7.0,59,150,9000000.0,17.7,1.0,7,0.104059,0.051364,...,0,0,0,0,0,1,0,0,0,0
13141811,2.0,7.0,49,136,18000000.0,6.5,1.0,6,0.107746,0.234971,...,0,0,0,0,0,1,0,0,0,0


In [12]:
final_train = final[final['is_applied'].notnull()]
final_test = final[final['is_applied'].isnull()]

In [13]:
data = final_train.drop('is_applied',axis=1)
target = final_train['is_applied']

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(data, target, test_size=0.3, random_state=0)